In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
import time
from selenium.webdriver.common.by import By
import logging
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import random

import json

In [ ]:
username = "huoerxiu@gmail.com"
password = "PURDUEcs"

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome(options=options)
actions = ActionChains(wd)

wd.maximize_window()
wd.switch_to.window(wd.current_window_handle)
wd.implicitly_wait(10)

wd.get('https://www.linkedin.com/')

wd.implicitly_wait(10)

In [ ]:
def random_sleep(start, end):
    time.sleep(random.randint(start, end))

def refresh_wd():
    # Define your Chrome options outside this function or ensure they are passed as parameters
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    # Create a new instance of Chrome
    wd = webdriver.Chrome(options=options)
    
    # Initialize action chains for possible future actions
    actions = ActionChains(wd)
    
    # Maximize the window to avoid elements being out of view
    wd.maximize_window()
    
    # Navigate to LinkedIn
    wd.get('https://www.linkedin.com/')

    return wd

def login(wd, user_name, password):
    try:
        # Use WebDriverWait for more reliable element handling
        username_input = WebDriverWait(wd, 10).until(
            EC.presence_of_element_located((By.ID, "session_key"))
        )
        username_input.clear()
        username_input.send_keys(user_name)
        
        password_input = WebDriverWait(wd, 10).until(
            EC.presence_of_element_located((By.ID, "session_password"))
        )
        password_input.clear()
        password_input.send_keys(password)

        print("Logging into your LinkedIn account!")
        
        # Locate and click the login button
        login_button = WebDriverWait(wd, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/main/section[1]/div/div/form/div[2]/button'))
        )
        login_button.click()
    except TimeoutException:
        print("Failed to login: Timeout while waiting for page elements.")
    except NoSuchElementException:
        print("Failed to login: Could not find one of the elements.")

def security_verification(wd):
    otp = input("You have been sent a verification code from LinkedIn via your email.\nPlease input that here: ")
    try:
        # Use WebDriverWait to wait for the OTP input field to become available
        otp_input = WebDriverWait(wd, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div/main/form/div[1]/input[15]'))
        )
        otp_input.clear()
        otp_input.send_keys(otp)

        # Locate and click the submit button for OTP
        submit_button = WebDriverWait(wd, 10).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div/main/form/div[2]/button'))
        )
        submit_button.click()
        print("Successfully Authenticated!")
    except TimeoutException:
        print("Authentication failed: Timeout while waiting for page elements.")
    except NoSuchElementException:
        print("Authentication failed: Could not find one of the elements.")

def search_query(wd, query):
    # Ensure the search input field is visible and clickable
    search_field = WebDriverWait(wd, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'search-global-typeahead__input'))
    )
    search_field.click()  # Focus on the search field

    # Introduce a random delay to mimic human typing speed
    random_sleep(1,2)

    # Send the search query
    search_field.send_keys(query)
    random_sleep()

    # Press ENTER to execute the search
    search_field.send_keys(Keys.ENTER)

def add_filters1(wd, location=None, current_company=None, past_company=None):
    navigate_to_people_results(wd)
    open_all_filters(wd)

    if current_company:
        apply_filter(wd, current_company, 'company', filter_index=0)
    if past_company:
        apply_filter(wd, past_company, 'company', filter_index=1)
    if location:
        apply_filter(wd, location, 'location')

    show_all_results(wd)
    return wd.current_url

def apply_filter(wd, filter_value, filter_type, filter_index=None):
    # Locate the "Add a filter" button based on the type and index (if applicable)
    filter_button_xpath = f"//*[text()='Add a {filter_type}']"
    filter_buttons = wd.find_elements(By.XPATH, filter_button_xpath)
    filter_button = filter_buttons[filter_index] if filter_index is not None else filter_buttons[0]

    wd.execute_script("arguments[0].scrollIntoView();", filter_button)
    filter_button.click()
    time.sleep(random.randint(1, 2))

    # Enter the filter value in the corresponding input field
    input_selector = f'input[placeholder="Add a {filter_type}"]'
    input_field = WebDriverWait(wd, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, input_selector))
    )
    input_field.click()
    input_field.send_keys(filter_value)
    time.sleep(random.randint(1, 2))

    # Wait for the listbox to appear and select the correct option
    listbox = WebDriverWait(wd, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'basic-typeahead__triggered-content'))
    )
    options = listbox.find_elements(By.XPATH, ".//div[@role='option']")
    for option in options:
        if filter_value in option.text:
            option.click()
            break
    time.sleep(random.randint(1, 2))

def navigate_to_people_results(wd):
    people_results_button = wd.find_element(By.XPATH, "//*[contains(text(), 'See all people results')]")
    wd.execute_script("arguments[0].style.border='2px solid red'", people_results_button)
    people_page_link = people_results_button.get_attribute("href")
    wd.get(people_page_link)
    time.sleep(random.randint(1, 2))

def open_all_filters(wd):
    all_filters = wd.find_element(By.CLASS_NAME, "relative.mr2")
    wd.execute_script("arguments[0].style.border='2px solid red'", all_filters)
    all_filters.click()
    random_sleep(1,2)

def industry_filter(wd, industry_list_num=0):
    # Navigate and click on all filters button
    navigate_and_click_filters(wd)

    # Scroll to and interact with the industry filter
    industry_filter_ul = WebDriverWait(wd, 10).until(
        EC.presence_of_element_located((By.XPATH, "//*[text()='Add an industry']/ancestor::ul[1]"))
    )
    wd.execute_script("arguments[0].scrollIntoView();", industry_filter_ul)

    # Select the industry based on provided index
    select_industry(wd, industry_filter_ul, industry_list_num)

    # Click show all results
    show_all_results(wd)

def navigate_and_click_filters(wd):
    all_filters = WebDriverWait(wd, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "relative.mr2"))
    )
    wd.execute_script("arguments[0].style.border='2px solid red'", all_filters)
    all_filters.click()
    time.sleep(random.randint(1, 2))

def select_industry(wd, industry_filter_ul, industry_list_num):
    li_elements = industry_filter_ul.find_elements(By.TAG_NAME, "li")
    if 0 <= industry_list_num < len(li_elements):
        li_element = li_elements[industry_list_num]
        input_checkbox = li_element.find_element(By.XPATH, ".//input[@type='checkbox']")
        if not input_checkbox.is_selected():
            wd.execute_script("arguments[0].click();", input_checkbox)
        print(f"Added industry option {industry_list_num}")
    else:
        print("Invalid industry index")

def show_all_results(wd):
    all_results = WebDriverWait(wd, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div/div/div[3]/div/button[2]'))
    )
    all_results.click()
    time.sleep(random.randint(1, 2))

def collect_links(wd, page_start, limit, total_results, unlimited=False):
    total_results = min(total_results, 1000)  # Limiting the total results to 1000
    names, curr_jobs, summarys, locations, links = [], [], [], [], []

    try:
        while True:
            # Check for "Retry Search" button at the start of each loop iteration
            handle_retry_search(wd)

            if page_start % 10 == 0:
                print(f"Scraping links on Page {page_start}")

            people_list = wd.find_elements(By.CLASS_NAME, "reusable-search__result-container")
            for person in people_list:
                try:
                    wd.execute_script("arguments[0].style.border='2px solid red'", person)
                    collect_person_info(person, names, curr_jobs, summarys, locations, links)
                except NoSuchElementException:
                    print("Required Element Not Found...Moving on")
                    wd.execute_script("arguments[0].style.border='2px solid green'", person)
                    continue

            random_sleep(2, 3)
            scroll_down(wd)

            if not process_next_page(wd, page_start, limit, unlimited):
                break

            page_start += 1

    except WebDriverException as e:
        print(f"An error occurred during the web scraping process: {str(e)}")

    return pd.DataFrame({
        'Name': names,
        'Current Job': curr_jobs,
        'Relevant Experience to Keyword': summarys,
        'Location': locations,
        'Profile Link': links
    })

def handle_retry_search(wd):
    try:
        retry_button = WebDriverWait(wd, 10).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(text(), 'Retry Search')]"))
        )
        print("Retry Search button found. Waiting before clicking.")
        random_sleep(42, 64)  # Wait for a random time between 42 and 64 seconds
        retry_button.click()
        print("Retry button clicked.")
    except TimeoutException:
        print("No 'Retry Search' button found, continuing with normal process.")

def collect_person_info(person, names, curr_jobs, summarys, locations, links):
    all_links = person.find_elements(By.TAG_NAME, 'a')
    name_text, curr_job_text, link_text, summary_text, location_text = extract_person_details(person, all_links)

    links.append(link_text)
    locations.append(location_text)
    summarys.append(summary_text)
    names.append(name_text)
    curr_jobs.append(curr_job_text)

def extract_person_details(person, all_links):
    name_element = person.find_element(By.CSS_SELECTOR, ".entity-result__title-text.t-16 a span[aria-hidden='true']")
    name_text = "LinkedIn Member" if "LinkedIn Member" in person.text else name_element.text

    curr_jobs = person.find_elements(By.CLASS_NAME, "entity-result__primary-subtitle.t-14.t-black.t-normal")
    curr_job_text = '"' + curr_jobs[0].text + '"' if curr_jobs else "Null"

    summary_elements = person.find_elements(By.TAG_NAME, 'p')
    summary_text = '"' + summary_elements[0].text + '"' if summary_elements else "Null"

    location_element = person.find_element(By.CSS_SELECTOR, ".entity-result__secondary-subtitle.t-14.t-normal")
    location_text = location_element.text

    link_text = "Null"
    for a in all_links:
        href = a.get_attribute('href')
        if href.startswith("https://www.linkedin.com/in") and not href.startswith("https://www.linkedin.com/in/ACo") and href not in all_links:
            link_text = href
            break

    return name_text, curr_job_text, link_text, summary_text, location_text

def scroll_down(wd):
    scroll_script = "window.scrollBy(0, 2000);"
    wd.execute_script(scroll_script)

def process_next_page(wd, page_start, limit, unlimited):
    next_page_button = wd.find_element(By.CLASS_NAME, "artdeco-pagination__button.artdeco-pagination__button--next")
    if next_page_button.is_enabled():
        next_page_button.click()
        random_sleep(1, 2)
        return True
    elif not unlimited and page_start == limit + 1:
        return False
    else:
        return False